In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import math
import torchvision
import matplotlib.pyplot as plt
from torchsummary import summary
import torchvision.transforms as transform
from torch.utils.data import Dataset
import glob
from torch.utils.data import DataLoader
import cv2
import os
import argparse
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.autograd import Variable
Tensor = torch.cuda.FloatTensor
import numpy as np
import pandas as pd
import torch.optim as optim 
from tqdm import tqdm
from PIL import Image


In [4]:
torch.manual_seed(111)
device=''
if torch.cuda.is_available():
      device = torch.device("cuda")
print(device)
#path = 
os.makedirs("images", exist_ok=True)


cuda


In [5]:
class Generator(nn.Module):
    def __init__(self):
        super (Generator,self).__init__()
        self.fc1 = nn.Linear(128, 256)
        self.relu1 = nn.LeakyReLU(0.25)
        self.fc2 = nn.Linear(256, 512)
        self.relu2 = nn.LeakyReLU(0.25)
        self.fc3 = nn.Linear(512,784)
        self.activation=nn.Tanh()
    
    def forward(self,x):
        x=self.relu1(self.fc1(x))
        x=self.relu2(self.fc2(x))
        x=self.fc3(x)
        return self.activation(x)



class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self). __init__()
        self.fc1 = nn.Linear(784, 1024)
        self.relu1 = nn.LeakyReLU(0.25)
        self.fc2 = nn.Linear(1024, 512)
        self.relu2 = nn.LeakyReLU(0.25)
        self.fc3 = nn.Linear(512,256)
        self.relu3 = nn.LeakyReLU(0.25)
        self.fc4 = nn.Linear(256,1)
        self.final= nn.Sigmoid()
    def forward(self,x):
        x=self.relu1(self.fc1(x))
        x=self.relu2(self.fc2(x))
        x=self.relu3(self.fc3(x))    
        x=self.fc4(x)
        return self.final(x)


model= Generator()
summary(model,(1,128))


model2= Discriminator()
summary(model2,(1,784))


Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 256]              33,024
├─LeakyReLU: 1-2                         [-1, 1, 256]              --
├─Linear: 1-3                            [-1, 1, 512]              131,584
├─LeakyReLU: 1-4                         [-1, 1, 512]              --
├─Linear: 1-5                            [-1, 1, 784]              402,192
├─Tanh: 1-6                              [-1, 1, 784]              --
Total params: 566,800
Trainable params: 566,800
Non-trainable params: 0
Total mult-adds (M): 0.57
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.16
Estimated Total Size (MB): 2.17
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 1024]             803,840
├─LeakyReLU: 1-2                         [-1, 1, 1024]             --
├─Linear: 1-3                            [-1, 1, 512]     

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 1024]             803,840
├─LeakyReLU: 1-2                         [-1, 1, 1024]             --
├─Linear: 1-3                            [-1, 1, 512]              524,800
├─LeakyReLU: 1-4                         [-1, 1, 512]              --
├─Linear: 1-5                            [-1, 1, 256]              131,328
├─LeakyReLU: 1-6                         [-1, 1, 256]              --
├─Linear: 1-7                            [-1, 1, 1]                257
├─Sigmoid: 1-8                           [-1, 1, 1]                --
Total params: 1,460,225
Trainable params: 1,460,225
Non-trainable params: 0
Total mult-adds (M): 1.46
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 5.57
Estimated Total Size (MB): 5.59

In [6]:
adversarial_loss = torch.nn.BCELoss().to(device)
generator = Generator().to(device)
discriminator=Discriminator().to(device)


In [7]:
'D:/OSU/Thesis/Fairness/MNIST/archive/trainingSet/trainingSet/'+str('0')

'D:/OSU/Thesis/Fairness/MNIST/archive/trainingSet/trainingSet/0'

In [8]:
df = pd.DataFrame(data={'path':['p1','p2','p3'],'label':[1,2,3]})
print(df)

  path  label
0   p1      1
1   p2      2
2   p3      3


In [9]:
len(os.listdir('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSample/trainingSample/0'))
len(os.listdir('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSet/trainingSet/0'))

4132

In [10]:
import pandas as pd
import numpy as np
import os

def generate_train_data():
    label_dict = dict([(j,i) for (i,j) in list(enumerate(os.listdir('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSet/trainingSet/')))])
    print(label_dict)

    paths = []
    label = []
    for i in tqdm(os.listdir('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSet/trainingSet/')):
        for j in os.listdir('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSet/trainingSet/'+str(i))[:100]:
            paths.append('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSet/trainingSet/'+str(i)+'/'+str(j))
            label.append(i)
    train_df = pd.DataFrame({"paths": paths, "label": label})  
    train_df = train_df.sample(frac=1)
    return train_df

def generate_test_data():
    paths = []
    label = []
    for i in tqdm(os.listdir('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSample/trainingSample/')):
        for j in os.listdir('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSample/trainingSample/'+str(i))[:50]:
            paths.append('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSample/trainingSample/'+str(i)+'/'+str(j))
            label.append(i)
    test_df = pd.DataFrame({"paths": paths, "label": label})
    return test_df



In [11]:
train_df = generate_train_data()
test_df = generate_test_data()
train_df.to_csv('./csv/train.csv')
test_df.to_csv('./csv/test.csv')

  0%|                                                                                                                                                                                      | 0/10 [00:00<?, ?it/s]

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 263.00it/s]


In [12]:
class CIFARDataSet(Dataset):
    def __init__(self, csv_file, transform,device):
        self.transform = transform
        self.df = csv_file
        self.device = device
        #self.standard_deviaton=std
        #self.mean2=mean
    def __len__(self):
        return len(self.df) 
            
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        label = label_dict[self.df.iloc[idx]['label']]
        label_onehot = np.zeros(10)
        label_onehot[label] = 1.
        path = self.df.iloc[idx]['paths']
        image = Image.open(path)
        image = self.transform(image)
        arr = np.random.randint(low = 0, high = 255, size = (300, 300, 3))
        im = Image.fromarray(arr.astype('uint8'))
        #image2=self.transform(image+torch.randn(tensor.size()) *std + mean2)
        sample = {
            "image": image.to(self.device),
            "noiseimage":im.to(self.device)
            #"noiseimage":image2.to(self.device),
            #to do add noised image
         #   "label": torch.from_numpy(label_onehot).to(self.device)
        }
        return sample

In [13]:
label_dict = dict([(j,i) for (i,j) in sorted(list(enumerate(os.listdir('D:/OSU/Thesis/Fairness/MNIST/archive/trainingSet/trainingSet/'))))])
print(label_dict)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


In [14]:
def get_train_dataloader(train_df, train_batch_size,device):
    traindataset = CIFARDataSet(train_df, transform=transform.Compose([
    transform.RandomCrop(32, padding=4),
    transform.RandomHorizontalFlip(),
    transform.ToTensor(),
    transform.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]), device=device)
    trainloader = DataLoader(traindataset, batch_size=train_batch_size,shuffle=True, num_workers=28)
    return trainloader

def get_test_dataloader(test_df,device):
    valdataset = CIFARDataSet(test_df, transform=transform.Compose([
        transform.ToTensor(),
        transform.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]), device=device)
    valloader = DataLoader(valdataset, batch_size=32, num_workers=28)
    return valloader

In [ ]:
batch_size = 32
iterations = 50
number_of_epochs = 1
learning_rate = 0.01
pretrained = True
i=4
dataloader= get_train_dataloader(train_df, batch_size,device)

C:\ProgramData\Anaconda3\lib\site-packages\torch\utils\data\dataloader.py:478: UserWarning: This DataLoader will create 28 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
dataloader= get_train_dataloader(train_df, batch_size,device)
testloader = get_test_dataloader(test_df,device)

optimzergen= torch.optim.SGD(generator.parameters(),lr=learning_rate)
optimzerdis= torch.optim.SGD(generator.parameters(),lr=learning_rate)

for batch in tqdm(dataloader):
    noise=batch['noise']
    image=batch['image']
     
    valid = Variable(Tensor(image.size(0), 1).fill_(1.0), requires_grad=False)
    fake = Variable(Tensor(noise.size(0), 1).fill_(0.0), requires_grad=False)
    trueimages = Variable(image.type(Tensor))
    
    #generator
    optimzergen.zero_grad()
    z = Variable(Tensor(np.random.normal(0, 1, (image.shape[0])))
    y_pred=generator(z)             
    g_loss = adversarial_loss(discriminator(gen_imgs), valid)
    g_loss.backward()
    optimizergen.step()
    
                 
    #Discrimintaor
    optimzerdis.zero_grad()
    real_loss = adversarial_loss(discriminator(trueimages), valid)
    fake_loss = adversarial_loss(discriminator(y_pred.detach()), fake)
    d_loss = (real_loss + fake_loss) / 2
    d_loss.backward()
    optimizerdis.step()

    with open('loss.txt', 'w+') as f:
                 for item in steady_latency:
                       f.write("%s\n" % item)
    
    print( "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )
  
        

        
